In [13]:
import pandas as pd 
import numpy as np


# Modèle collaboratif > Travail préliminaire

Ce notebook présente le travail préliminaire de features engineering pour parvenir à un jeux de données qui corresponde à ce que l'on souhaite obtenir en entrée de notre algorithme collaboratif. 

Nous obtenons premièrement une matrice utilisateurs / items dont les éléments "rating" sont le nombre de transaction users>items ayant eu lieu (ou non)

A partir de cette matrice, nous obtenons une nouvelle matrice "fc_matrix".
Fc matrix est un "applatissement de la matrice utilisateurs/items. Elle doit contenir trois colonnes "userID","itemID" et les éléments de transaction "rating".

Les codes des utilisateurs et des items sont transcris en nombres simples et compréhensible pour faciliter l'analyse et garder une confidentialité des données.

In [2]:
df = pd.read_csv('fc_data.csv')

In [120]:
print (df.columns[:])

Index(['Unnamed: 0', 'InvoiceNo', 'StockCode', 'Description', 'Quantity',
       'InvoiceDate', 'UnitPrice', 'CustomerID', 'Country', 'Country_bis'],
      dtype='object')


In [3]:
def plot_value_counts(col_name,df):       
    
    values_count = pd.DataFrame(df[col_name].dropna().value_counts())
    print (values_count.shape)
    values_count.columns = ['count']
    # convert the index column into a regular column.
    values_count[col_name] = [ str(i) for i in values_count.index ]
    # add a column with the percentage of each data point to the sum of all data points.
    values_count['percent'] = values_count['count'].div(values_count['count'].sum()).multiply(100).round(2)
    # change the order of the columns.
    values_count = values_count.reindex([col_name,'count','percent'],axis=1)
    values_count.reset_index(drop=True,inplace=True)
    return (values_count)

In [4]:
df_cf = df[['InvoiceNo','CustomerID','Quantity','StockCode','UnitPrice']]

In [5]:
ind_cust = df_cf.columns.get_loc('CustomerID')
ind_q = df_cf.columns.get_loc('Quantity')
ind_price = df_cf.columns.get_loc('UnitPrice')
ind_stock = df_cf.columns.get_loc('StockCode')
ind_numb = df_cf.columns.get_loc('InvoiceNo')

valeurs_cust = plot_value_counts('CustomerID',df=df_cf)
valeurs_stock = plot_value_counts('StockCode',df=df_cf)


(4256, 1)
(3908, 1)


In [6]:
print (valeurs_cust.iloc[0:10,:])
valeurs_cust = valeurs_cust.drop(valeurs_cust.index[0])

  CustomerID   count  percent
0       Nnan  130931    25.29
1   N17841.0    7183     1.39
2   N14911.0    5313     1.03
3   N14096.0    5095     0.98
4   N12748.0    4519     0.87
5   N14606.0    2547     0.49
6   N15311.0    2350     0.45
7   N14646.0    2054     0.40
8   N13089.0    1810     0.35
9   N13263.0    1672     0.32


In [7]:
# ce module à pour objectif de tronquer le nombre d'utilisateur ainsi que le nombre de produit de telle sorte à ne garder 
# que les n_u et n_p utilisateurs et produits les plus actifs (resp les plus populaires)
n_u = 200
n_p = 200

valeurs_stock_t = valeurs_stock.iloc[0:n_p,:]
valeurs_cust_t = valeurs_cust.iloc[0:n_u,:]




In [8]:
# associer à chaque couple produit/consommateur un Rui

# supprimer les colonnes inutiles 
valeurs_cust_rui = valeurs_cust_t.drop(columns = ['count','percent'])

# créer un vecteur numpy 
vect = np.zeros(valeurs_cust_t.shape[0])
for i in range (valeurs_stock_t.shape[0]):
    valeurs_cust_rui['{}'.format(valeurs_stock_t.iloc[i,0])] = vect[:]
    
valeurs_cust_pui = valeurs_cust_rui.copy()
    
# on itére sur les produits 
for i in range (valeurs_stock_t.shape[0]):
    
    df_cf1 = df_cf[(df_cf.iloc[:,ind_stock] == valeurs_stock_t.iloc[i,0])]
  
    for j in range (valeurs_cust_rui.shape[0]):
        df_ij = df_cf1[(df_cf1.iloc[:,ind_cust] == valeurs_cust_rui.iloc[j,0])]#&
                 
        valeurs_cust_rui.iloc[j,i+1] = df_ij.shape[0] 
        
    



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


In [9]:
# associer aux utilisateurs une valeur numérique 
vect = np.zeros(valeurs_cust_rui.shape[0])
for i in range (valeurs_cust_rui.shape[0]):
    vect[i] = i
# sauvegarder un df_pandas pour retrouver le numéro correspondant
d = {'number': valeurs_cust_rui.iloc[:,0], 'value': vect[:]}
df_users = pd.DataFrame(data=d)

# associer aux items une valeur numérique 
vect = np.zeros(valeurs_stock_t.shape[0])
for i in range (valeurs_stock_t.shape[0]):
    vect[i] = i
# sauvegarder un df_pandas pour retrouver le numéro correspondant
d = {'number': valeurs_stock_t.iloc[:,0], 'value': vect[:]}
df_items = pd.DataFrame(data=d)

# créer la matrice utilisateur 
d = {'it0': df_users.iloc[:,1]}
for i in range (valeurs_stock_t.shape[0]-1):
   
    d.update({'it{}'.format(i+1): df_users.iloc[:,1]})
    users_matrix = pd.DataFrame(data=d)   
    
print (users_matrix)

# créer la matrice item
d = {'it0': df_items.iloc[:,1]}
for i in range (valeurs_cust_rui.shape[0]-1):
    d.update({'it{}'.format(i+1): df_items.iloc[:,1]})
    items_matrix = pd.DataFrame(data=d)       
    
    
print (items_matrix)

 

    

       it0    it1    it2    it3    it4    it5    it6    it7    it8    it9  \
1      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
3      2.0    2.0    2.0    2.0    2.0    2.0    2.0    2.0    2.0    2.0   
4      3.0    3.0    3.0    3.0    3.0    3.0    3.0    3.0    3.0    3.0   
5      4.0    4.0    4.0    4.0    4.0    4.0    4.0    4.0    4.0    4.0   
6      5.0    5.0    5.0    5.0    5.0    5.0    5.0    5.0    5.0    5.0   
7      6.0    6.0    6.0    6.0    6.0    6.0    6.0    6.0    6.0    6.0   
8      7.0    7.0    7.0    7.0    7.0    7.0    7.0    7.0    7.0    7.0   
9      8.0    8.0    8.0    8.0    8.0    8.0    8.0    8.0    8.0    8.0   
10     9.0    9.0    9.0    9.0    9.0    9.0    9.0    9.0    9.0    9.0   
11    10.0   10.0   10.0   10.0   10.0   10.0   10.0   10.0   10.0   10.0   
12    11.0   11.0   11.0   11.0   11.0   11.0   11.0   11.0   11.0   11.0   

       it0    it1    it2    it3    it4    it5    it6    it7    it8    it9  \
0      0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
1      1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0    1.0   
2      2.0    2.0    2.0    2.0    2.0    2.0    2.0    2.0    2.0    2.0   
3      3.0    3.0    3.0    3.0    3.0    3.0    3.0    3.0    3.0    3.0   
4      4.0    4.0    4.0    4.0    4.0    4.0    4.0    4.0    4.0    4.0   
5      5.0    5.0    5.0    5.0    5.0    5.0    5.0    5.0    5.0    5.0   
6      6.0    6.0    6.0    6.0    6.0    6.0    6.0    6.0    6.0    6.0   
7      7.0    7.0    7.0    7.0    7.0    7.0    7.0    7.0    7.0    7.0   
8      8.0    8.0    8.0    8.0    8.0    8.0    8.0    8.0    8.0    8.0   
9      9.0    9.0    9.0    9.0    9.0    9.0    9.0    9.0    9.0    9.0   
10    10.0   10.0   10.0   10.0   10.0   10.0   10.0   10.0   10.0   10.0   
11    11.0   11.0   11.0   11.0   11.0   11.0   11.0   11.0   11.0   11.0   

In [10]:
# vecteur note
rating = np.matrix(valeurs_cust_rui.iloc[:,1::])
rating = rating.flatten().transpose()
print (rating.shape)

# vecteur user
um = np.matrix(users_matrix.iloc[:,:])
um = um.flatten().transpose()
print (um.shape)

# vecteur item
it = np.matrix(items_matrix.iloc[:,:])
it = it.transpose().flatten().transpose()
print (it.shape)





(40000, 1)
(40000, 1)
(40000, 1)


In [11]:
vect = np.zeros(it.shape)

d = {'try': vect[:,0]}
fc_matrix = pd.DataFrame(data=d)
fc_matrix['itemID'] = it[:,0]
fc_matrix['userID'] = um[:,0]
fc_matrix['rating'] = rating[:,0]

In [12]:
# save 
fc_matrix.to_csv("fc_matrix_red200.csv")
df_users.to_csv("fc_users_conv_red200.csv")
df_items.to_csv("fc_items_conv_red200.csv")